In [179]:

"""
This file is containing a class, that perform all extraction modules for 
Hovagim Bakardjian system that serves for classification of 

"""

import aseegg as ag
import numpy as np
import pandas as pd
from bieg import ICAManager
import matplotlib.pyplot as plt
import math
import scipy.signal as sig

pre_data_path = "../data/subject1/SSVEP_8Hz_Trial1_SUBJ1.MAT"
pre_data_path2 = "../data/subject1/SSVEP_14Hz_Trial5_SUBJ1.MAT"
pre_data_path3 = "../data/subject1/SSVEP_28Hz_Trial3_SUBJ1.MAT"

class BakardjianSystem(object):
    
    def __init__(self,path,seconds,components_to_exclude = [0,14,16,22,24,27,28,127],s_08Hz=np.zeros(1),
                s_14Hz = np.zeros(1),s_28Hz = np.zeros(1),es_08Hz = np.zeros(1),
                es_14Hz = np.zeros(1),es_28Hz = np.zeros(1)):
        self.path = path
        self.seconds = seconds
        self.ica_file = ICAManager(input_path=self.path,method='fastica')
        self.components_to_exclude = components_to_exclude
        self.s_08Hz = s_08Hz
        self.s_14Hz = s_14Hz
        self.s_28Hz = s_28Hz
        self.es_08Hz = es_08Hz
        self.es_14Hz = es_14Hz
        self.es_28Hz = es_28Hz
        self.window_length = 15

    #Module 1: Blind Source Separation/ICA 
    def bss_ica(self):
        
        self.ica_file = ICAManager(input_path=self.path,method='fastica')
        self.ica_file.load_matlab_data()
        self.ica_file.extract_components()
        self.ica_file.exclude_ica_components(components_to_exclude=self.components_to_exclude)
        return self.ica_file
    
    #Module 2: Narrow-band filters
        
    def bank_of_filters(self):
    
        #Electrodes of occipital region
        
        ica_file = self.ica_file
        
        beg = 5*256
        end = 20*256

        electrode_O1 = ica_file.data[15][beg:end]
        electrode_0z = ica_file.data[23][beg:end]
        electrode_O2 = ica_file.data[28][beg:end]

        prefiltered = [electrode_0z,electrode_O1,electrode_O2]

        #Signals filtered:

        self.s_08Hz = np.zeros((3,len(electrode_O1)))
        self.s_14Hz = np.zeros((3,len(electrode_O1)))
        self.s_28Hz = np.zeros((3,len(electrode_O1)))

        for n in range(0,3):
            signal_filtered_08Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=7.9,czestOdciecia2=8.1,czestProbkowania=256)
            signal_filtered_14Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=13.9,czestOdciecia2=14.1,czestProbkowania=256)
            signal_filtered_28Hz = ag.pasmowoprzepustowy(prefiltered[n],czestOdciecia1=27.9,czestOdciecia2=28.1,czestProbkowania=256)
            self.s_08Hz[n] = signal_filtered_08Hz
            self.s_14Hz[n] = signal_filtered_14Hz
            self.s_28Hz[n] = signal_filtered_28Hz
            
        return self.s_08Hz, self.s_14Hz, self.s_28Hz
        
    #Module3: Variance analyzer
        
    def variance_analyzer(self):
        for z in [self.s_08Hz,self.s_14Hz,self.s_28Hz]:
            for n in range(0,z.shape[0]):
                z[n] = np.square(z[n])
                z[n] = np.sqrt(z[n])
        return self.s_08Hz, self.s_14Hz, self.s_28Hz
    
    #Module 4: Smoothing procedure

    def smoother(self):        
        for z in [self.s_08Hz,self.s_14Hz,self.s_28Hz]:
            for n in range(0,z.shape[0]):
                z[n] = sig.savgol_filter(z[n],polyorder=2,window_length=self.window_length)
        return self.s_08Hz,self.s_14Hz,self.s_28Hz

    #Module5: Integrator
    
    def integrator(self):
        self.es_08Hz = np.divide((self.s_08Hz[0] + self.s_08Hz[1] + self.s_08Hz[2]),self.s_08Hz.shape[0])
        self.es_14Hz = np.divide((self.s_14Hz[0] + self.s_14Hz[1] + self.s_14Hz[2]),self.s_14Hz.shape[0])
        self.es_28Hz = np.divide((self.s_28Hz[0] + self.s_28Hz[1] + self.s_28Hz[2]),self.s_28Hz.shape[0]) 
        return self.es_08Hz,self.es_14Hz,self.es_28Hz
    
    #Module6: Normalization
    
    def normalizer(self):
        self.s_08Hz = np.sum(self.es_08Hz)/np.sum((np.sum(self.s_08Hz[0]),
                                     np.sum(self.s_08Hz[1]),np.sum(self.s_08Hz[2])))
        self.s_14Hz = np.sum(self.es_14Hz)/np.sum((np.sum(self.s_14Hz[0]),
                                     np.sum(self.s_14Hz[1]),np.sum(self.s_14Hz[2])))
        self.s_28Hz = np.sum(self.es_28Hz)/np.sum((np.sum(self.s_28Hz[0]),
                                     np.sum(self.s_28Hz[1]),np.sum(self.s_28Hz[2])))
        return self.s_08Hz,self.s_14Hz,self.s_28Hz

    #Bakardjian system, all functions at once
    
    def bak_system(self):
        self.bss_ica()
        self.bank_of_filters()
        self.variance_analyzer()
        self.smoother()
        self.integrator()
        self.normalizer()
        return self.s_08Hz, self.s_14Hz, self.s_28Hz 
        

    

In [182]:
bs = BakardjianSystem(pre_data_path3,seconds=15)

In [183]:
bs.bak_system()

Creating RawArray with float64 data, n_channels=128, n_times=6379
    Range : 0 ... 6378 =      0.000 ...    24.914 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected


/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.


[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


Using all PCA components: 128
<ICA  |  raw data decomposition, fit (fastica): 2052 samples, 128 components, channels used: "eeg">
Creating RawArray with float64 data, n_channels=128, n_times=6379
    Range : 0 ... 6378 =      0.000 ...    24.914 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected


/usr/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.


[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


Using all PCA components: 128
<ICA  |  raw data decomposition, fit (fastica): 2052 samples, 128 components, channels used: "eeg">
Transforming to ICA space (128 components)
Zeroing out 8 ICA components


(0.33333333333333337, 0.33333333333333337, 0.33333333333333331)